# Recommendation System

In [10]:
import pandas as pd
import numpy as np
import warnings 
from sklearn.metrics.pairwise import cosine_similarity
warnings.simplefilter("ignore")

In [2]:
df =pd.read_csv("anime.csv")
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [4]:
df.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [5]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [6]:
df["genre"].mode()

0    Hentai
Name: genre, dtype: object

In [7]:
df["genre"]=df["genre"].fillna(df["genre"].mode()[0])
df["type"]=df["type"].fillna(df["type"].mode()[0])
df["rating"]=df["rating"].fillna(0)

In [8]:
df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [9]:
df_0=pd.pivot_table(data=df,index='anime_id',columns='genre',values='rating')
df_0.fillna(0,inplace=True)
df_0

genre,Action,"Action, Adventure","Action, Adventure, Cars, Comedy, Sci-Fi, Shounen","Action, Adventure, Cars, Mecha, Sci-Fi, Shounen, Sports","Action, Adventure, Cars, Sci-Fi","Action, Adventure, Comedy","Action, Adventure, Comedy, Demons, Drama, Ecchi, Horror, Mystery, Romance, Sci-Fi","Action, Adventure, Comedy, Demons, Fantasy, Magic","Action, Adventure, Comedy, Demons, Fantasy, Magic, Romance, Shounen, Supernatural","Action, Adventure, Comedy, Demons, Fantasy, Martial Arts, Shounen, Super Power",...,Slice of Life,"Slice of Life, Space","Slice of Life, Supernatural",Space,Sports,"Super Power, Supernatural, Vampire",Supernatural,Thriller,Vampire,Yaoi
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Recommendation System and Evalution

In [12]:
similar=cosine_similarity(df_0)
similar

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
similar_df=pd.DataFrame(similar)
similar_df

,0,1,2,3,4,5,6,7,8,9,...,12284,12285,12286,12287,12288,12289,12290,12291,12292,12293
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
12290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
similar_df.index=df.anime_id.unique()
similar_df.columns=df.anime_id.unique()
similar_df

,32281,5114,28977,9253,9969,32935,11061,820,15335,15417,...,26031,34399,10368,9352,5541,9316,5543,5621,6133,26081
32281,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5114,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
5621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
np.fill_diagonal(similar,0)
similar_df.idxmax()[0:15]

32281     8800
5114     32281
28977      868
9253     32281
9969     12413
32935    30296
11061    32281
820      29283
15335      264
15417    32281
4181     27787
28851    20581
918       3176
2904       534
28891      793
dtype: int64

In [16]:
df[(df.anime_id==28891) | (df.anime_id==793)]

,anime_id,name,genre,type,episodes,rating,members
14,28891,Haikyuu!! Second Season,"Comedy, Drama, School, Shounen, Sports",TV,25,8.93,179342
484,793,xxxHOLiC Movie: Manatsu no Yoru no Yume,"Comedy, Drama, Mystery, Psychological, Superna...",Movie,1,8.04,41547


In [17]:
df[(df.anime_id==918) | (df.anime_id==793)]

,anime_id,name,genre,type,episodes,rating,members
12,918,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,201,9.04,336376
484,793,xxxHOLiC Movie: Manatsu no Yoru no Yume,"Comedy, Drama, Mystery, Psychological, Superna...",Movie,1,8.04,41547


In [18]:
df[(df.anime_id==32281) | (df.anime_id==264)]

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
162,264,Hajime no Ippo: Champion Road,"Comedy, Shounen, Sports",Special,1,8.39,47840


In [21]:
ppl_1=df[df['anime_id']==264]
ppl_2=df[df['anime_id']==32281]

In [22]:
ppl_1

,anime_id,name,genre,type,episodes,rating,members
162,264,Hajime no Ippo: Champion Road,"Comedy, Shounen, Sports",Special,1,8.39,47840


In [23]:
ppl_2

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630


In [24]:
ppl_1.name

162    Hajime no Ippo: Champion Road
Name: name, dtype: object

In [25]:
ppl_2.name

0    Kimi no Na wa.
Name: name, dtype: object

In [26]:

user_1 = list(set(ppl_1.name))
user_2 = list(set(ppl_2.name))
print(user_1)
print(user_2)

['Hajime no Ippo: Champion Road']
['Kimi no Na wa.']


In [28]:
user_1 = list(set(ppl_1.name))
user_2 = list(set(ppl_2.name))

for name_of_anime in user_1:
  if name_of_anime not in user_2:
    print("Recommendation of the ANIME to watch Next : ", name_of_anime)

Recommendation of the ANIME to watch Next :  Hajime no Ippo: Champion Road


In [ ]:
for x in similar_df.index:
    sim_user = similar_df.idxmax(axis=1)[x]
    print (x,sim_user)
    ist_user=df[df['anime_id']== x]
    tnd_user=df[df['anime_id']== sim_user]
    user_1 = list(set(ppl_1.name))
    user_2 = list(set(ppl_2.name))

                                  
    for name_of_anime in user_1:
        if name_of_anime not in user_2:
            print("Recommendation of anime to watch for userid:",sim_user , name_of_anime)
    for name_of_anime in user_2:
        if name_of_anime not in user_1:
            print("Recommendation of anime to watch for userid:", x,name_of_anime)

32281 8800
Recommendation of anime to watch for userid: 8800 Hajime no Ippo: Champion Road
Recommendation of anime to watch for userid: 32281 Kimi no Na wa.
5114 32281
Recommendation of anime to watch for userid: 32281 Hajime no Ippo: Champion Road
Recommendation of anime to watch for userid: 5114 Kimi no Na wa.
28977 868
Recommendation of anime to watch for userid: 868 Hajime no Ippo: Champion Road
Recommendation of anime to watch for userid: 28977 Kimi no Na wa.
9253 32281
Recommendation of anime to watch for userid: 32281 Hajime no Ippo: Champion Road
Recommendation of anime to watch for userid: 9253 Kimi no Na wa.
9969 12413
Recommendation of anime to watch for userid: 12413 Hajime no Ippo: Champion Road
Recommendation of anime to watch for userid: 9969 Kimi no Na wa.
32935 30296
Recommendation of anime to watch for userid: 30296 Hajime no Ippo: Champion Road
Recommendation of anime to watch for userid: 32935 Kimi no Na wa.
11061 32281
Recommendation of anime to watch for userid: 3

## interview Questions 

1. #### User-based collaborative filtering
it is like finding people with similar tastes and seeing what they recommend. If you and your friends all love action movies, the system might recommend other action movies you haven't seen yet.

#### Item-based collaborative filtering 
it is like looking at what movies your friends have in common. If you and your friends all love a particular movie, the system might recommend other movies that people who liked that movie also enjoyed.

2. Collaborative filtering is a recommendation technique that predicts a user's preferences based on the preferences and behaviors of other users. It operates on the idea that users who have agreed in the past will likely agree in the future.